In [46]:
import ipywidgets as widgets
from IPython.display import display

# GUI elements

header = widgets.HBox([widgets.Label(value="Sound Cloud Search")])

song_text = widgets.Text(
    value='',
    placeholder='Enter song name..',
    description='Song:',
    disabled=False
)

artist_text = widgets.Text(
    value='',
    placeholder='Enter artist name..',
    description='Artist:',
    disabled=False
)

generate_button = widgets.Button(
    description='Generate Cloud',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Generate Word Cloud',
    icon='cloud' # (FontAwesome names without the `fa-` prefix)
)

clear_button = widgets.Button(
    description='Clear Search',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Clear Search',
    icon='times-circle' # (FontAwesome names without the `fa-` prefix)
)

output = widgets.Output()

In [47]:
# Button handling

def generate_button_clicked(b):
    with output:
        print("[DEBUG - generate_button_clicked]\nSong:", song_text.value, "\nArtist:", artist_text.value, "\n")        
def clear_button_clicked(b):
    song_text.value = ''
    artist_text.value = ''
    with output:
        print("[DEBUG - clear_clicked()]\nCleared 'song_text' and 'artist_text'\n")
        
generate_button.on_click(generate_button_clicked)
clear_button.on_click(clear_clicked)

In [48]:
# Display GUI
display(header, song_text, artist_text, generate_button, clear_button, output)

Text(value='', description='Song:', placeholder='Enter song name..')

Text(value='', description='Artist:', placeholder='Enter artist name..')

Button(button_style='success', description='Generate Cloud', icon='cloud', style=ButtonStyle(), tooltip='Gener…

Button(button_style='danger', description='Clear Search', icon='times-circle', style=ButtonStyle(), tooltip='C…

Output()

In [49]:
# Canvas

from ipycanvas import Canvas

# creating canvas
canvas = Canvas(width=400, height=200)

# setting font size
canvas.font = '32px serif'

# stroke_text or fill_text(text, x, y, max_width=None)
canvas.fill_text('Testing ipycanvas', 50, 50)
canvas.fill_text('Testing ipycanvas', 50, 90)

# call canvas
canvas

Canvas(height=200, width=400)